In [1]:
# Directory containing the sample videos
DATA_DIRECTORY = "../sample_data"

In [2]:
from pathlib import Path
from enum import Enum

class FileExtension(Enum):
    MOV = ".mov"
    MP4 = ".mp4"

class DataDirectory:

    def __init__(self, data_directory: str):
        self.data_directory = Path(data_directory)

    def get_video_folders(self) -> list[Path]:
        self.video_folders = [folder for folder in self.data_directory.iterdir() if folder.is_dir()]
        return self.video_folders
    
    def get_all_video_files(self, folder_name: str, file_extension: FileExtension = FileExtension.MOV) -> list[Path]:
        folder_path = self.data_directory / folder_name
        return [file for file in folder_path.iterdir() if file.is_file() and file.suffix == file_extension.value or file_extension.value.upper()]
    
    def get_video_file(self, folder_name: str, index: int=0, file_extension: FileExtension = FileExtension.MOV) -> Path:
        video_files = self.get_all_video_files(folder_name, file_extension)
        return video_files[index]


In [3]:
file_slno = 0
folder_slno = 0

data_directory = DataDirectory(DATA_DIRECTORY)
folder_name = data_directory.get_video_folders()[folder_slno].name
video_file_path = data_directory.get_all_video_files(folder_name=folder_name)[file_slno]
video_file_path


PosixPath('../sample_data/Phiphi/IMG_8146.MOV')

In [4]:
import ffmpeg
from geopy.geocoders import Nominatim
from pydantic import BaseModel
from typing import Annotated
from datetime import datetime

class VideoMetaData(BaseModel):
    duration: Annotated[int, "time in seconds"]
    created: Annotated[datetime, "created date"]
    modified: Annotated[datetime, "modified date"]
    location: Annotated[str, "location details"]
    framerate: Annotated[int, "frame rate"]


class VideoFile:

    def __init__(self, filepath:Path) -> None:
        self.filepath = str(filepath)
    
    def get_metadata(self):
        probe = ffmpeg.probe(self.filepath)
        
        metadata = {}
        
        # Get video stream information
        video_info = next((stream for stream in probe['streams'] 
                          if stream['codec_type'] == 'video'), None)
        if video_info:
            metadata.update({
                'width': video_info.get('width'),
                'height': video_info.get('height'),
                'duration': video_info.get('duration'),
                'avg_frame_rate': video_info.get('avg_frame_rate'),
            })
            
        # Get format information
        format_info = probe.get('format', {})
        # Get metadata tags if they exist
        tags = format_info.get('tags', {})
        if tags:
            metadata['tags'] = tags
            
            # GPS data is often stored in tags
            gps_data = {k: v for k, v in tags.items() 
                       if k.lower().startswith(('gps', 'location', 'geo'))}
            if gps_data:
                metadata['gps_data'] = gps_data
        return metadata
    
    @property
    def parse_location_iso6709(location_iso: str):
        location_iso = location_iso.strip('/')
        lat = float(location_iso[0:8])
        lon = float(location_iso[8:17])
        alt = float(location_iso[17:])
        return {"latitude": lat, "longitude": lon, "altitude_m": alt}
    
    @property
    def get_place_name(latitude: float, longitude: float) -> str:
        geolocator = Nominatim(user_agent="geoapi")
        location = geolocator.reverse((latitude, longitude), exactly_one=True)
        return location.address if location else "Unknown Location"




In [5]:
video_file = VideoFile(filepath=video_file_path)
video_file.get_metadata()

{'width': 1920,
 'height': 1080,
 'duration': '5.000000',
 'avg_frame_rate': '30/1',
 'tags': {'major_brand': 'qt  ',
  'minor_version': '0',
  'compatible_brands': 'qt  ',
  'creation_time': '2024-05-04T12:22:25.000000Z',
  'com.apple.quicktime.location.accuracy.horizontal': '3.535534',
  'com.apple.quicktime.cinematic-video': '',
  'com.apple.quicktime.location.ISO6709': '+07.8849+098.4135+006.341/',
  'com.apple.quicktime.make': 'Apple',
  'com.apple.quicktime.model': 'iPhone 13',
  'com.apple.quicktime.software': '17.3',
  'com.apple.quicktime.creationdate': '2024-02-11T10:04:22+0700'}}